In [10]:
import sys
import os
from os import path
# 部署的路径
sys.path.append('/root/bdrisk/risk_project')
# 堡垒机的路径
sys.path.append(path.dirname(path.dirname(path.dirname(os.getcwd()))))
# 这是log文件的存放路径
root_path = path.dirname(path.dirname(path.dirname(os.getcwd())))
from risk_models import *
from risk_models import _name_FT_I_DTL_SEA_PRE_RECORDED, _name_FT_I_DTL_SEA_LIST, _name_FT_I_DTL_OTR_PRE_RECORDED, _name_FT_I_DTL_OTR_LIST, _name_FT_E_DTL_SEA_PRE_RECORDED, _name_FT_E_DTL_SEA_LIST, _name_FT_E_DTL_OTR_PRE_RECORDED, _name_FT_E_DTL_OTR_LIST, _name_DIM_FTZ_CORP, _name_DIM_TRADER


class DisplayDp1(object):
    def __init__(self, child_task_id, base_time):
        self.child_task_id = child_task_id
        Risk_logger(child_task_id = self.child_task_id, exec_status=None)

        # 参数读取
        self.base_time = base_time.replace('-','')
        self.base_time = self.base_time.replace('/','')
        self.base_time = self.base_time[0:8]

    def date_standard(self, df):
        """
        des: 填充缺失的时间
        input df['col']
        output df_series
        """
        b = []
        date_begin = df.min()
        date_end = df.max()
        a = pd.date_range(date_begin, date_end, freq='MS', normalize=True)
        for i in a:
            i = str(i)
            i = datetime.datetime.strptime(i, '%Y-%m-%d %H:%M:%S')
            b.append(i)
        date_serie = pd.DataFrame(b)
        date_serie = date_serie.rename(columns={0: 'STANDARD_DATE'})
        return date_serie

    def model_dp1(self):
        # 计算统计起始时间（当前时间的去年1月份）
        start_time = str( int(self.base_time[0:4]) - 1 ) + '0101'

        # 进口报关单货值
        sql_I = '''
        SELECT 'I' AS IE_FLAG,
        T1.TIME AS DATETIME,
        SUM(T1.ENTRY_NUM) AS ENTRY_NUM,
        SUM(T1.RMB)/100000000 AS RMB,
        SUM(T1.TRADE_NUM) AS TRADE_NUM,
        '报关单' as type,
        (SELECT count(distinct corp_uid) FROM {}) as corp_num
        FROM 
        (SELECT TO_DATE(SUBSTR(A.APL_DATE_KEY,1,6),'YYYYMM') AS TIME , 
        COUNT(DISTINCT A.PRE_ENTRY_NO) AS ENTRY_NUM,SUM(B.GOODS_GROSS_VALUE_RMB) AS RMB ,COUNT(DISTINCT A.TRADE_CODE) AS TRADE_NUM
        FROM {} A 
        JOIN {} B ON A.ID =B.HEAD_ID
        WHERE A.APL_DATE_KEY >= {} AND A.APL_DATE_KEY <= {}  AND A.CUSTOMS_CODE ='2249'
        AND A.TRADE_CODE IN (select distinct trade_code from {} where trade_code_scc in (select uni_sc_id FROM {}))
        GROUP BY SUBSTR(A.APL_DATE_KEY,1,6)
        union all
        SELECT TO_DATE(SUBSTR(C.APL_DATE_KEY,1,6),'YYYYMM') AS TIME , 
        COUNT(DISTINCT C.PRE_ENTRY_NO) AS ENTRY_NUM,SUM(D.GOODS_GROSS_VALUE_RMB) AS RMB,COUNT(DISTINCT C.TRADE_CODE) AS TRADE_NUM
        FROM {} C 
        JOIN {} D ON C.ID =D.HEAD_ID
        WHERE C.APL_DATE_KEY >= {} AND C.APL_DATE_KEY <= {}  AND C.CUSTOMS_CODE ='2249'
        AND C.TRADE_CODE IN (select distinct trade_code from {} where trade_code_scc in (select uni_sc_id FROM {}))
        GROUP BY SUBSTR(C.APL_DATE_KEY,1,6)) T1
        GROUP BY T1.TIME
        ORDER BY TIME
        '''.format(_name_DIM_FTZ_CORP, _name_FT_I_DTL_SEA_PRE_RECORDED, _name_FT_I_DTL_SEA_LIST, start_time, self.base_time, _name_DIM_TRADER, _name_DIM_FTZ_CORP, 
                   _name_FT_I_DTL_OTR_PRE_RECORDED, _name_FT_I_DTL_OTR_LIST, start_time, self.base_time, _name_DIM_TRADER, _name_DIM_FTZ_CORP)
        t1 = Read_Oracle().read_oracle(sql=sql_I, database='dbdw')
        
        # 时间字段标准化
        t1['DATETIME'] = self.date_standard(t1['DATETIME'])
        
        # 出口报关单货值
        sql_E = '''
        SELECT 'E' AS IE_FLAG,
        T1.TIME AS DATETIME,
        SUM(T1.ENTRY_NUM) AS ENTRY_NUM,
        SUM(T1.RMB)/100000000 AS RMB,
        SUM(T1.TRADE_NUM) AS TRADE_NUM,
        '报关单' as type,
        (SELECT count(distinct corp_uid) FROM {}) as corp_num
        FROM 
        (SELECT TO_DATE(SUBSTR(A.APL_DATE_KEY,1,6),'YYYYMM') AS TIME , 
        COUNT(DISTINCT A.PRE_ENTRY_NO) AS ENTRY_NUM,SUM(B.GOODS_GROSS_VALUE_RMB) AS RMB ,COUNT(DISTINCT A.TRADE_CODE) AS TRADE_NUM
        FROM {} A 
        JOIN {} B ON A.ID =B.HEAD_ID
        WHERE A.APL_DATE_KEY >= {} AND A.APL_DATE_KEY <= {}  AND A.CUSTOMS_CODE ='2249'
        AND A.TRADE_CODE IN (select distinct trade_code from {} where trade_code_scc in (select uni_sc_id FROM {}))
        GROUP BY SUBSTR(A.APL_DATE_KEY,1,6)
        union all
        SELECT TO_dATE(SUBSTR(C.APL_DATE_KEY,1,6),'YYYYMM') AS TIME , 
        COUNT(DISTINCT C.PRE_ENTRY_NO) AS ENTRY_NUM,SUM(D.GOODS_GROSS_VALUE_RMB) AS RMB,COUNT(DISTINCT C.TRADE_CODE) AS TRADE_NUM
        FROM {} C 
        JOIN {} D ON C.ID =D.HEAD_ID
        WHERE C.APL_DATE_KEY >= {} AND C.APL_DATE_KEY <= {}  AND C.CUSTOMS_CODE ='2249'
        AND C.TRADE_CODE IN (select distinct trade_code from {} where trade_code_scc in (select uni_sc_id FROM {}))
        GROUP BY SUBSTR(C.APL_DATE_KEY,1,6)) T1
        GROUP BY T1.TIME
        ORDER BY TIME
        '''.format(_name_DIM_FTZ_CORP, _name_FT_E_DTL_SEA_PRE_RECORDED, _name_FT_E_DTL_SEA_LIST, start_time, self.base_time, _name_DIM_TRADER, _name_DIM_FTZ_CORP, 
                   _name_FT_E_DTL_OTR_PRE_RECORDED, _name_FT_E_DTL_OTR_LIST, start_time, self.base_time, _name_DIM_TRADER, _name_DIM_FTZ_CORP)
        t2 = Read_Oracle().read_oracle(sql=sql_E, database='dbdw')
        
        # 时间字段标准化
        t2['DATETIME'] = self.date_standard(t2['DATETIME'])
        
        # 合并进出口数据并添加ID字段
        df_1 = pd.concat([t1, t2])
        df_1.reset_index(inplace=True)
        df_1['ID'] = range(len(df_1))
        df_1 = df_1[['ID', 'IE_FLAG', 'DATETIME', 'ENTRY_NUM', 'RMB', 'TRADE_NUM', 'TYPE', 'CORP_NUM']]
        
        Write_Oracle().write_oracle('BD_RISK_DISPLAY_MAIN',df_1,org_code = None, alarm=None)
        return print(df_1)

    def run_model_dp1(self):
        try:
            self.model_dp1()
            exec_status = 1
        except:
            logger.exception('model execution error')
            exec_status = 0
        finally:
            Risk_logger(child_task_id = self.child_task_id, exec_status=exec_status).write_log()


if __name__ == '__main__':
    # 读取传入的 child_task_id，仅适用于py调用,先备注掉
    # child_task_id = sys.argv[1]
    child_task_id = '0001_0001'
    org_code, param_json, base_time = read_log_table(child_task_id)
    DisplayDp1(child_task_id, base_time).run_model_dp1()



2021-05-26 11:09:45.843 | INFO     | risk_models.config.read_config.read_func:read_oracle:82 - Read Table successfully! , Total read time spent 0.120s
2021-05-26 11:11:10.107 | INFO     | risk_models.config.read_config.read_func:read_oracle:82 - Read Table successfully! , Total read time spent 84.22s
2021-05-26 11:13:19.959 | INFO     | risk_models.config.read_config.read_func:read_oracle:82 - Read Table successfully! , Total read time spent 129.8s
2021-05-26 11:13:20.116 | INFO     | risk_models.config.read_config.read_func:read_oracle:82 - Read Table successfully! , Total read time spent 0.102s
2021-05-26 11:13:20.273 | INFO     | risk_models.config.write_config.write_func:write_oracle:141 - Processing... Writing 30 rows into database
2021-05-26 11:13:20.307 | INFO     | risk_models.config.write_config.write_func:write_oracle:148 - Insert data into BD_RISK_DISPLAY_MAIN successfully! Total write time spent 0.299s
2021-05-26 11:13:20.341 | INFO     | risk_models.config.log_config.log_f

    ID IE_FLAG   DATETIME  ENTRY_NUM        RMB  TRADE_NUM TYPE  CORP_NUM
0    0       I 2020-01-01       2716  54.936127         84  报关单        71
1    1       I 2020-02-01       2315  42.153535         81  报关单        71
2    2       I 2020-03-01       3776  27.002489         84  报关单        71
3    3       I 2020-04-01       3557  23.564106         84  报关单        71
4    4       I 2020-05-01       2829  15.226430         83  报关单        71
5    5       I 2020-06-01       3115  24.080007         88  报关单        71
6    6       I 2020-07-01       3610  37.542975         92  报关单        71
7    7       I 2020-08-01       2996  43.441228         94  报关单        71
8    8       I 2020-09-01       3454  60.042294         93  报关单        71
9    9       I 2020-10-01       3063  55.276551         90  报关单        71
10  10       I 2020-11-01       3265  33.927280         87  报关单        71
11  11       I 2020-12-01       4807  42.316309         87  报关单        71
12  12       I 2021-01-01       3952  

2021-05-26 11:13:21.383 | INFO     | risk_models.config.write_config.write_log:write_oracle:52 - Updated child_task_id: 0001_0001 log msg into BD_RISK_MODEL_LOG successfully!
